Imports

In [1]:
import pandas as pd
import numpy as np
import sys
import os


from src.image_tools import ImageTools
from src.load_image import load_image_to_numpy_array
pd.options.mode.chained_assignment = None  # default='warn'

Paths and constants

In [2]:
"""
WORKING_DIR = "C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/"
GT_PATH = WORKING_DIR+"data/data_yoloformat/test_annotations_for_programming"

IMAGE_PATH = "data/data_yoloformat/test_images_for_programming"
#yolo
PRED_PATH = WORKING_DIR+"results/yolov4_testresults/annotations/absolute_format"
#faster-rcnn
"""

WORKING_DIR = os.getcwd()


GT_PATH = "examples/test_labels/"
GT = os.listdir(GT_PATH)
IMAGE_PATH = "examples/test_im/"
IM = os.listdir(IMAGE_PATH)
PRED_PATH = "examples/test_pred/"
PRED = os.listdir(PRED_PATH)

In [125]:
class EvaluationTools:
    def __init__(self, gt_path, im_path, pred_path, iou_threshold):
        self.im_path = im_path
        self.gt_path = gt_path
        self.pred_path = pred_path
        self.iou_threshold = iou_threshold
        self.names = ['classification', 'x', 'y', 'w', 'h']
        self.names_pred = ['classification', 'confidence', 'x', 'y', 'w', 'h']

        self.im_w, self.im_h = self.get_image_info()


    def get_bbox(self):
        """
        Convert and return bonding box from path to pandas df (in x, y, h, w)
        """
        gt = pd.read_csv(self.gt_path, sep = ' ', names = self.names)
        pred = pd.read_csv(self.pred_path, sep = ' ', header=None).drop(6, axis=1)
        pred.columns = self.names_pred

        return pred, self.yolo_to_absolute(gt)


    def absolute_to_yolo(self, bbox):
        """
        NOT USED
        Convert bbox format from (xmin, ymin, w, h) to yolo
        """
        n = (bbox.shape[0])
        converted_bbox = bbox.copy()
        for i in range(n):
            converted_bbox['x'][i] = bbox['x'][i] / self.im_w
            converted_bbox['y'][i] = bbox['y'][i] / self.im_h
            converted_bbox['w'][i] = bbox['w'][i] / self.im_w
            converted_bbox['h'][i] = bbox['h'][i] / self.im_h

        return converted_bbox

    def absolute_to_coordinates(self, bbox):
        """
        convert bbox format from (xmin, ymin, w, h) to (xmin, ymin, xmax, ymax).
        Param: bbox:
                numpy array shape [n,4]
        returns:
                numpy array shape [n,4]
        """
        converted_bbox = bbox.copy()
        converted_bbox[:, 2] = bbox[:, 0] + bbox[:, 2]
        converted_bbox[:, 3] = bbox[:, 1] + bbox[:, 3]

        return converted_bbox

    def yolo_to_absolute(self, bbox):
        """
        Convert bbox format from yolo to (xmin, ymin, w, h)
        """
        n = (bbox.shape[0])
        converted_bbox = bbox.copy()
        
        for i in range(n):
            converted_bbox['x'][i] = (bbox['x'][i] - 0.5*bbox['w'][i]) * self.im_w
            converted_bbox['y'][i] = (bbox['y'][i] - 0.5*bbox['h'][i]) * self.im_h
            converted_bbox['w'][i] = bbox['w'][i] * self.im_w
            converted_bbox['h'][i] = bbox['h'][i] * self.im_h

        return converted_bbox

    def get_image_info(self):
        """
        Fetch image height and width
        """
        im = load_image_to_numpy_array(self.im_path)
        im_w = im.shape[1]
        im_h = im.shape[0]
        return im_w, im_h
    
    def get_iou(self, pred, gt):
        """
        Calculate intersection over union (IoU) between all bounding boxes in image.
        Also calculates total pixel area of each bounding box

        param:
            
        returns:
        iou: float(0,1)
        area_bb1: float
        area_bb2: float
        """
        #remove values class/conf values and converte to coordinate form (pascalVOC)
        bb1 = self.absolute_to_coordinates(np.array(pred)[:, 2:])
        #add extra axis for calculation
        bb1 = bb1[:, None]
        #remove class value and convert to coordinate form (pascalVOC)
        bb2 = self.absolute_to_coordinates(np.round(np.array(gt)[:, 1:], 0))

        #calculation...
        low = np.s_[...,:2]
        high = np.s_[..., 2:]

        bb1[high] += 1; bb2[high] += 1
        
        #intersect
        intrs = (np.maximum(0,np.minimum(bb1[high],bb2[high])
                            -np.maximum(bb1[low],bb2[low]))).prod(-1)
        #iou
        area_pred = (bb1[high]-bb1[low]).prod(-1)
        area_gt = (bb2[high]-bb2[low]).prod(-1)
        iou = intrs / (area_pred + area_gt -intrs + 1e-16)
        #print(area_pred)
        #print(area_gt)
        return iou, area_pred
    
    def find_valid_detections(self, iou, iou_threshold):
        """
        find number TP, FP, FN 
        """
        #print(iou)
        closest_box = np.max(iou, axis=1)
        #print('closest_box: ', closest_box)
        valid_boxes = closest_box[closest_box >= iou_threshold]
        tp = valid_boxes.shape[0]
        fp = closest_box.shape[0] - valid_boxes.shape[0]

        
        closest_gt = np.max(iou, axis=0)
        #print(closest_gt)
        valid_gt = closest_gt[closest_gt >= iou_threshold]
        #print('valid_gt: ', valid_gt)
        fn = closest_gt.shape[0] - valid_gt.shape[0]
        
        #print('false positives: ', fp)
        #print('false negatives: ', fn)
        tp = valid_boxes.shape[0]
        
        return valid_boxes, tp, fp, fn
        
    
    
    def number_of_detections(self, valid_boxes, area_pred):
        print(valid_boxes.shape)
        print(area_pred.shape)
        area = np.ravel(area_pred)
        print(area.shape)
        #iterators
        pred_small = 0
        pred_med = 0
        pred_large = 0
        pred_tot = 0
        #thresholds
        small_thresh = 32**2
        med_thresh = 96**2

         #= self.find_valid_detections(iou, iou_threshold)
         
        
        for i in range(valid_boxes.shape[0]):
            
            #print('area: ', area.shape)
            pred_tot += 1
           
            if area[i] < small_thresh:
                pred_small += 1
            elif area[i] >= small_thresh and area[i] < med_thresh:
                pred_med += 1
            elif area[i] >= med_thresh:
                pred_large += 1
            else:
                raise Exception('Value for bbox area is invalid')

        
        return pred_tot, pred_small, pred_med, pred_large


main()


(5,)
(5, 1)
(5,)
(1,)
(1, 1)
(1,)
(1,)
(1, 1)
(1,)
precision:  1.0
recall:  1.0


In [121]:
def recall(tp, fn):
    return tp / (tp + fn)

def precision(tp, fp):
    return tp / (tp + fp)
    



    



def main():
    TP = 0
    FP = 0
    FN = 0
    PRED_LARGE = 0
    PRED_MEDIUM = 0
    PRED_SMALL = 0
    PRED_TOTAL = 0
    THRESH = 0.5
  
    #for test_index in range(len(IM)):
    for test_index in range(3):
        #test_index = 3
        test = EvaluationTools(gt_path = GT_PATH+GT[test_index], im_path = IMAGE_PATH+IM[test_index],
                                pred_path=PRED_PATH+PRED[test_index], iou_threshold=THRESH)
        
        pred, gt = test.get_bbox()

        iou, area_pred = test.get_iou(pred, gt)
        valid_boxes, tp, fp, fn = test.find_valid_detections(iou, THRESH)
        
        
        pred_tot, pred_small, pred_med, pred_large = test.number_of_detections(valid_boxes, area_pred)
        TP += tp
        FP += fp
        FN += fn
        PRED_LARGE += pred_large
        PRED_MEDIUM += pred_med
        PRED_SMALL += pred_small
        PRED_TOTAL += pred_tot
    
    rc = recall(TP, FN)
    pc = precision(TP, FP)


    print('precision: ', pc)
    print('recall: ', rc)


    #print(pred_tot)
    #print(pred_small, pred_med, pred_large)
    #ious = test.slow_iou(pred, gt)
    #print(iou)

if __name__ == '__main__':
    main()

(5, 1)
(1, 1)
(1, 1)
precision:  1.0
recall:  1.0


NameError: name 'area_bb1' is not defined